<a href="https://colab.research.google.com/github/amillam/Coursera_Capstone/blob/master/Guerra_de_Vecindarios_Sem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Importing required libraries
import numpy as np
import pandas as pd

from geopy.geocoders import Nominatim
try:
    import geocoder
except:
    !pip install geocoder
    import geocoder

import requests
from bs4 import BeautifulSoup

try:
    import folium
except:
    !pip install folium
    import folium
    
from sklearn.cluster import KMeans

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 4.3 MB/s 


In [38]:
g = geocoder.arcgis('Bangalore, India')
blr_lat = g.latlng[0]
blr_lng = g.latlng[1]
print("The Latitude and Longitude of Bangalore is {} and {}".format(blr_lat, blr_lng))

The Latitude and Longitude of Bangalore is 12.966180000000065 and 77.58690000000007


In [7]:
#Scraping the webpage for list of localities
neig = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Bangalore").text

In [8]:
soup = BeautifulSoup(neig, 'html.parser')
#Creating a list to store neighborhood data
neighborhoodlist = []
for i in soup.find_all('div', class_='mw-category')[0].find_all('a'):
    neighborhoodlist.append(i.text)

#Creating a dataframe from the list
neig_df = pd.DataFrame({"Locality": neighborhoodlist})
neig_df.head()

,Locality
0,"Agara, Bangalore"
1,Arekere
2,Banashankari
3,Banaswadi
4,Basavanagudi


In [9]:
#Shape of dataframe neig_df
neig_df.shape

(59, 1)

In [10]:
#Defining a function to get the location of the localities
def get_location(localities):
    g = geocoder.arcgis('{}, Bangalore, India'.format(localities))
    get_latlng = g.latlng
    return get_latlng

In [11]:
co_ordinates = []
for i in neig_df["Locality"].tolist():
    co_ordinates.append(get_location(i))
print(co_ordinates)

[[12.84292000000005, 77.48582000000005], [12.885670000000061, 77.59673000000004], [12.922310000000039, 77.56988000000007], [13.019264919035423, 77.6556029157363], [12.938980000000072, 77.57137000000006], [12.986740000000054, 77.54389000000003], [12.882460000000037, 77.62477000000007], [12.927340000000072, 77.67169000000007], [12.966180000000065, 77.58690000000007], [12.966180000000065, 77.58690000000007], [12.817510000000027, 77.67881000000006], [12.794010000000071, 77.70015000000006], [12.967160000000035, 77.63340000000005], [12.966180000000065, 77.58690000000007], [12.943290000000047, 77.65602000000007], [12.845410000000072, 77.66434000000004], [12.998940000000061, 77.61276000000004], [12.942790000000059, 77.54122000000007], [13.026410000000055, 77.62437000000006], [13.049810000000036, 77.58903000000004], [12.932660000000055, 77.54675000000003], [13.026220192074469, 77.72030129486366], [12.912220000000048, 77.64470000000006], [13.030060000000049, 77.49526000000003], [12.9234400000000

In [12]:
#Creating a dataframe from the list of location
co_ordinates_df = pd.DataFrame(co_ordinates, columns=['Latitudes', 'Longitudes'])

In [13]:
#Adding co-ordinated to neig_df dataframe
neig_df["Latitudes"] = co_ordinates_df["Latitudes"]
neig_df["Longitudes"] = co_ordinates_df["Longitudes"]

In [14]:
neig_df.head()

,Locality,Latitudes,Longitudes
0,"Agara, Bangalore",12.842920,77.485820
1,Arekere,12.885670,77.596730
2,Banashankari,12.922310,77.569880
3,Banaswadi,13.019265,77.655603
4,Basavanagudi,12.938980,77.571370


In [16]:
#Creating a map
blr_map = folium.Map(location=[blr_lat, blr_lng],zoom_start=11)

#adding markers to the map for localities
#marker for Bangalore
folium.Marker([blr_lat, blr_lng], popup='<i>Bangalore</i>', tooltip="Click to see").add_to(blr_map)

#markers for localities
for latitude,longitude,name in zip(neig_df["Latitudes"], neig_df["Longitudes"], neig_df["Locality"]):
    folium.CircleMarker(
        [latitude, longitude],
        radius=6,
        color='blue',
        popup=name,
        fill=True,
        fill_color='#3186ff'
    ).add_to(blr_map)

blr_map

In [17]:
#Foursquare Credentials
CLIENT_ID = 'JTB4R2ZERJU1QIVN1L4DXTEHZZS3ALDRVPDITI5KSV45D0DG'
CLIENT_SECRET = 'ICQ5C1WJOIFWHALH01K3XKDN4UFX3Q5PT3I4ZBNVW3P1SVKD'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + "CLIENT_ID")
print('CLIENT_SECRET:' + "CLIENT_SECRET")

Your credentails:
CLIENT_ID: CLIENT_ID
CLIENT_SECRET:CLIENT_SECRET


In [18]:
#Getting the top 100 venues in each locality
radius = 2000
LIMIT = 100

venues = []

for lat, lng, locality in zip(neig_df["Latitudes"], neig_df["Longitudes"], neig_df["Locality"]):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
    results = requests.get(url).json()['response']['groups'][0]['items']

    for venue in results:
        venues.append((locality, lat, lng, venue['venue']['name'], venue['venue']['location']['lat'], venue['venue']['location']['lng'], venue['venue']['categories'][0]['name']))

In [19]:
venues[0]

('Agara, Bangalore',
 12.84292000000005,
 77.48582000000005,
 'Aagara Lake',
 12.849695,
 77.48907,
 'Lake')

In [20]:
#Convert the venue list into dataframe
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Locality', 'Latitude', 'Longitude', 'Venue name', 'Venue Lat', 'Venue Lng', 'Venue Category']
venues_df.head()

,Locality,Latitude,Longitude,Venue name,Venue Lat,Venue Lng,Venue Category
0,"Agara, Bangalore",12.84292,77.48582,Aagara Lake,12.849695,77.489070,Lake
1,Arekere,12.88567,77.59673,Decathlon Sports India Pvt Ltd,12.887513,77.597712,Sporting Goods Shop
2,Arekere,12.88567,77.59673,Natural Ice Cream,12.892188,77.598222,Ice Cream Shop
3,Arekere,12.88567,77.59673,Swensens,12.876071,77.595542,Ice Cream Shop
4,Arekere,12.88567,77.59673,Cinepolis,12.876119,77.595455,Multiplex


In [21]:
#Number of venues for each Locality
venues_df.groupby(['Locality']).count()

,Latitude,Longitude,Venue name,Venue Lat,Venue Lng,Venue Category
Locality,,,,,,
"Agara, Bangalore",1,1,1,1,1,1
Arekere,81,81,81,81,81,81
Banashankari,100,100,100,100,100,100
Banaswadi,67,67,67,67,67,67
Basavanagudi,100,100,100,100,100,100
Basaveshwaranagar,57,57,57,57,57,57
"Begur, Bangalore",12,12,12,12,12,12
Bellandur,78,78,78,78,78,78
Bengaluru Pete,100,100,100,100,100,100


In [22]:
#Getting the unique categories
print('There are {} unique categries.'.format(len(venues_df['Venue Category'])))

There are 3398 unique categries.


In [23]:
#List of categories
print('Total number of unique catefories are {}'.format(len(venues_df['Venue Category'].unique().tolist())))
#First 10 categories
venues_df['Venue Category'].unique().tolist()#[:10]

Total number of unique catefories are 206


['Lake',
 'Sporting Goods Shop',
 'Ice Cream Shop',
 'Multiplex',
 'BBQ Joint',
 'Indian Restaurant',
 'Lounge',
 'Beer Garden',
 'Bowling Alley',
 'Pizza Place',
 'Café',
 'Chinese Restaurant',
 'Liquor Store',
 'Restaurant',
 'Eastern European Restaurant',
 'Brewery',
 'South Indian Restaurant',
 'Office',
 'Sandwich Place',
 'Department Store',
 'Shopping Mall',
 'Fast Food Restaurant',
 'Rajasthani Restaurant',
 'Middle Eastern Restaurant',
 'American Restaurant',
 'Burger Joint',
 'General Entertainment',
 'Coffee Shop',
 'Italian Restaurant',
 'Supermarket',
 'Dumpling Restaurant',
 'Dive Bar',
 'Clothing Store',
 'Electronics Store',
 'Food Court',
 'Badminton Court',
 'Snack Place',
 'Movie Theater',
 'Furniture / Home Store',
 'Cosmetics Shop',
 'Bookstore',
 'Vegetarian / Vegan Restaurant',
 'Buffet',
 'Kebab Restaurant',
 'Cafeteria',
 'Grocery Store',
 'Asian Restaurant',
 'Gift Shop',
 'Andhra Restaurant',
 'Photography Studio',
 'Donut Shop',
 'Indian Chinese Restaurant',

In [24]:
#one hot encoding
blr_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

blr_onehot['Locality'] = venues_df['Locality']

#move the locality column to the front
blr_onehot = blr_onehot[ [ 'Locality' ] + [ col for col in blr_onehot.columns if col!='Locality' ] ]
blr_onehot.head()

,Locality,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,...,Trail,Train Station,Travel & Transport,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Agara, Bangalore",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Arekere,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Arekere,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arekere,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Arekere,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
blr_grouped = blr_onehot.groupby(['Locality']).mean().reset_index()
print(blr_grouped.shape)
blr_grouped.head()

(58, 207)


,Locality,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,...,Trail,Train Station,Travel & Transport,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Agara, Bangalore",0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
1,Arekere,0.0,0.0,0.0,0.0,0.012346,0.012346,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.024691,0.0,0.0,0.0,0.00,0.0
2,Banashankari,0.0,0.0,0.0,0.0,0.000000,0.000000,0.01,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.0
3,Banaswadi,0.0,0.0,0.0,0.0,0.000000,0.014925,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.029851,0.0,0.0,0.0,0.00,0.0
4,Basavanagudi,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.01,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.0


In [42]:
#numbers of localities having Italian Restaurants
len(blr_grouped[blr_grouped['American Restaurant'] > 0])

12

In [44]:
blr_american = blr_grouped[['Locality', 'American Restaurant']]
blr_american.head()

,Locality,American Restaurant
0,"Agara, Bangalore",0.000000
1,Arekere,0.012346
2,Banashankari,0.000000
3,Banaswadi,0.000000
4,Basavanagudi,0.000000


In [54]:
#K-means clustering
cluster = 3

#Dataframe for clustering
blr_clustering = blr_american.drop(['Locality'], 1)

#run K-means clustering
k_means = KMeans(init="k-means++", n_clusters=cluster, n_init=12).fit(blr_clustering)

#getting the labels for first 10 locality 
print(k_means.labels_[0:10])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


[0 1 0 0 0 0 0 0 1 1]


In [55]:
#Creating a of blr_italian dataframe
blr_labels = blr_american.copy()

#addring label to blr_labels
blr_labels["Cluster Label"] = k_means.labels_

blr_labels.head()

,Locality,American Restaurant,Cluster Label
0,"Agara, Bangalore",0.000000,0
1,Arekere,0.012346,1
2,Banashankari,0.000000,0
3,Banaswadi,0.000000,0
4,Basavanagudi,0.000000,0


In [56]:
#Merging the blr_labels and neig_df dataframes to get the latitude and longitudes for each locality
blr_labels = blr_labels.join(neig_df.set_index('Locality'), on='Locality')
blr_labels.head()

,Locality,American Restaurant,Cluster Label,Latitudes,Longitudes
0,"Agara, Bangalore",0.000000,0,12.842920,77.485820
1,Arekere,0.012346,1,12.885670,77.596730
2,Banashankari,0.000000,0,12.922310,77.569880
3,Banaswadi,0.000000,0,13.019265,77.655603
4,Basavanagudi,0.000000,0,12.938980,77.571370


In [57]:
#Grouping the localities according to their Cluster Labels
blr_labels.sort_values(["Cluster Label"], inplace=True)
blr_labels.head()

,Locality,American Restaurant,Cluster Label,Latitudes,Longitudes
0,"Agara, Bangalore",0.0,0,12.84292,77.48582
27,"Jayanagar, Bangalore",0.0,0,12.92868,77.58270
29,Kamakya,0.0,0,12.92376,77.55362
30,Kettohalli,0.0,0,12.90669,77.40468
31,"Kodihalli, Bangalore",0.0,0,12.96621,77.64980


In [58]:
#Plot the cluster on map
cluster_map = folium.Map(location=[blr_lat, blr_lng],zoom_start=11)

#marker for Bangalore
folium.Marker([blr_lat, blr_lng], popup='<i>Bangalore</i>', tooltip="Click to see").add_to(cluster_map)

#Getting the colors for the clusters
col = ['red', 'green', 'blue']

#markers for localities
for latitude,longitude,name,clus in zip(blr_labels["Latitudes"], blr_labels["Longitudes"], blr_labels["Locality"], blr_labels["Cluster Label"]):
    label = folium.Popup(name + ' - Cluster ' + str(clus))
    folium.CircleMarker(
        [latitude, longitude],
        radius=6,
        color=col[clus],
        popup=label,
        fill=False,
        fill_color=col[clus],
        fill_opacity=0.3
    ).add_to(cluster_map)
       
cluster_map

In [60]:
#First Cluster
cluster_1 = blr_labels[blr_labels['Cluster Label'] == 0]
print("There are {} localities in cluster-1".format(cluster_1.shape[0]))
mean_presence_1 = cluster_1['American Restaurant'].mean()
print("The mean occurence of American restaurant in cluster-1 is {0:.2f}".format(mean_presence_1))
cluster_1

There are 46 localities in cluster-1
The mean occurence of American restaurant in cluster-1 is 0.00


,Locality,American Restaurant,Cluster Label,Latitudes,Longitudes
0,"Agara, Bangalore",0.0,0,12.842920,77.485820
27,"Jayanagar, Bangalore",0.0,0,12.928680,77.582700
29,Kamakya,0.0,0,12.923760,77.553620
30,Kettohalli,0.0,0,12.906690,77.404680
31,"Kodihalli, Bangalore",0.0,0,12.966210,77.649800
32,Konanakunte,0.0,0,12.882290,77.569230
33,Koramangala,0.0,0,12.920040,77.625460
36,Madiwala,0.0,0,12.920540,77.620970
37,Magadi,0.0,0,12.995014,77.443150
38,Mahadevapura,0.0,0,12.994090,77.666330


In [61]:
#Second Cluster
cluster_2 = blr_labels[blr_labels['Cluster Label'] == 1]
print("There are {} localities in cluster-2".format(cluster_2.shape[0]))
mean_presence_2 = cluster_2['American Restaurant'].mean()
print("The mean occurence of American restaurant in cluster-2 is {0:.2f}".format(mean_presence_2))
cluster_2

There are 10 localities in cluster-2
The mean occurence of American restaurant in cluster-2 is 0.01


,Locality,American Restaurant,Cluster Label,Latitudes,Longitudes
8,Bengaluru Pete,0.010000,1,12.96618,77.58690
34,Krishnarajapura,0.014493,1,13.00039,77.68368
35,Kundalahalli,0.013158,1,12.96751,77.71500
49,Shivajinagar,0.010000,1,12.98718,77.60398
13,Dhobi Ghat (Bangalore),0.010000,1,12.96618,77.58690
52,UB City,0.010000,1,12.97147,77.59570
39,Majestic (Bangalore),0.010000,1,12.97759,77.57256
1,Arekere,0.012346,1,12.88567,77.59673
9,Bidadi,0.010000,1,12.96618,77.58690
19,HSR Layout,0.014493,1,12.91222,77.64470


In [62]:
#Third Cluster
cluster_3 = blr_labels[blr_labels['Cluster Label'] == 2]
print("There are {} localities in cluster-3".format(cluster_3.shape[0]))
mean_presence_3 = cluster_3['American Restaurant'].mean()
print("The mean occurence of American restaurant in cluster-3 is {0:.2f}".format(mean_presence_3))
cluster_3

There are 2 localities in cluster-3
The mean occurence of American restaurant in cluster-3 is 0.04


,Locality,American Restaurant,Cluster Label,Latitudes,Longitudes
56,Yelahanka,0.038462,2,13.09931,77.59259
54,Varthur,0.037037,2,12.94349,77.74701
